In [ ]:
import os
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

import pandas as pd


data_dir = Path("data")

performance_csv = data_dir / "cloth_competition_performance.csv"


df = pd.read_csv(performance_csv)
# df = pd.read_csv(performance_csv, index_col=False)
# df.reset_index(drop=True, inplace=True)

df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import matplotlib as mpl
from matplotlib.colors import ListedColormap

def plot_examples(colormaps):
    """
    Helper function to plot data with associated colormap.
    """
    np.random.seed(19680801)
    data = np.random.randn(30, 30)
    n = len(colormaps)
    fig, axs = plt.subplots(1, n, figsize=(n * 2 + 2, 3),
                            layout='constrained', squeeze=False)
    for [ax, cmap] in zip(axs.flat, colormaps):
        psm = ax.pcolormesh(data, cmap=cmap, rasterized=True, vmin=-4, vmax=4)
        fig.colorbar(psm, ax=ax)
    plt.show()


def muted_colormap(name: str, amount=0.4):

    original = mpl.colormaps[name]
    muted = ListedColormap(original(np.linspace(0.0, amount, 128)))
    return muted
    
name = "Greens"
plot_examples([mpl.colormaps[name], muted_colormap(name)])

In [ ]:
latex_table = df.to_latex()
print(latex_table)

In [ ]:
styled_df = df.copy()

# Apply the desired format to float values
styled_df = styled_df.style.format("{:.2f}", subset=pd.IndexSlice[:, df.dtypes == float])

# Replace '%' with '\%' and '&' with '\&'
styled_df = styled_df.format(lambda x: x.replace('%', r'\%').replace('&', r'\&') if isinstance(x, str) else x)

# Convert to LaTeX and print the result
latex_table = styled_df.to_latex()
print(latex_table)

In [ ]:
import pandas as pd

# Sample DataFrame

# Apply the desired format to float values
styled_df = df.copy()
styled_df = styled_df.style.format("{:.2f}", subset=pd.IndexSlice[:, df.dtypes == float])

# # Replace '%' with '\%' and '&' with '\&'
# styled_df = styled_df.format(lambda x: x.replace('%', r'\%').replace('&', r'\&') if isinstance(x, str) else x)

styled_df = styled_df.background_gradient(cmap=muted_colormap("Purples"), subset="Grasp success rate", axis=0)  # axis=0 for column-wise gradient


styled_df = styled_df.background_gradient(cmap=muted_colormap("Blues"), 
                                          subset="Coverage for successful grasps", 
                                          axis=0)
                                        # vmin=0.57,
                                        # vmax=0.75)  # axis=0 for column-wise gradient

styled_df = styled_df.background_gradient(cmap=muted_colormap("Greens"), 
                                          subset="Coverage", 
                                          axis=0)  # axis=0 for column-wise gradient

# Replace '%' with '\%' and '&' with '\&'
styled_df = styled_df.format(lambda x: x.replace('%', r'\%').replace('&', r'\&') if isinstance(x, str) else x)

# Make the first row bold
# styled_df = styled_df.set_properties(**{'font-weight': 'bold'}, subset=pd.IndexSlice[0, :])


# Apply background gradient to numeric columns
#numeric_cols = df.select_dtypes(include=['number']).columns
#styled_df = styled_df.background_gradient(cmap="viridis", subset=numeric_cols, axis=0)  # axis=0 for column-wise gradient

# Convert to LaTeX and print the result
# latex_table = styled_df.to_latex()
# print(latex_table)

from IPython.display import HTML

html_table = styled_df.to_html()  # Convert DataFrame to HTML
HTML(html_table)

In [ ]:
latex_table = styled_df.to_latex(hrules=False, column_format="lllccc", convert_css=True)
# latex_table = latex_table.replace(r"\background-color#", r"\cellcolor[HTML]{")
# latex_table = latex_table.replace(r" \color#000000 ", "}") 

print(latex_table) # MANUAL EDIT 0.6 to 0.60

In [ ]:
df

In [ ]:
def is_on_pareto_front(grasp_success, coverage):
    """
    Check if a given point is on the Pareto front.
    """
    for other_grasp_success, other_coverage in zip(df['Grasp success rate'], df['Coverage for successful grasps']):
        if (grasp_success < other_grasp_success and coverage < other_coverage):
            return False
    return True

frontier = []

for row in df.itertuples():
    grasp_success_rate = row._3  # Access the value in 'Grasp success rate'
    coverage = row._4  # Access the value in 'Coverage for successful grasps'
    # Do something with grasp_success_rate and coverage
    print(is_on_pareto_front(grasp_success_rate, coverage))
  
    if is_on_pareto_front(grasp_success_rate, coverage):
        frontier.append((grasp_success_rate, coverage))




frontier = np.array(frontier)

# sort the frontier according to grasp success rate
frontier = frontier[np.argsort(frontier[:, 0])]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# plt.figure(figsize=(6, 8))
plt.figure(figsize=(6, 5))


# Plot the frontier
# plt.plot(frontier[:, 0], frontier[:, 1], color='#41faa4', linestyle='dashed')
plt.plot(frontier[:, 0], frontier[:, 1], color='#95d0fc', linestyle='dashed')

ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Create the scatterplot
# plt.scatter(df['Grasp success rate'], df['Coverage for successful grasps'], zorder=2.5, color="#14c774", marker='o')
scatter = plt.scatter(df['Grasp success rate'], df['Coverage for successful grasps'], zorder=2.5, color="#1390f0", marker='o')

# annotations = [r'$1^{st}$', r'$2^{nd}$', r'$3^{rd}$'] # ['1st', '2nd', '3rd']
# annotations = ['1st', '2nd', '3rd']

# for i, txt in enumerate(annotations):
#     plt.annotate(txt, (scatter.get_offsets()[i][0], scatter.get_offsets()[i][1]), textcoords="offset points", xytext=(-1,4), ha='center')


plt.xlim(0, 1)
plt.ylim(0.3, 1)

# Add labels and title
plt.xlabel('Grasp Success Rate')
plt.ylabel('Coverage for successful grasps')
plt.title('Trade-off between grasp success and effectiveness')

# plt.gca().set_aspect('equal', adjustable='box')

plt.savefig("pareto.svg", format="svg") 

# Show the plot
plt.show()